In [ ]:
import tweepy
import json
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import gensim.downloader as gensimapi
from nltk.tokenize import TreebankWordTokenizer
from emoji import UNICODE_EMOJI
import unicodedata as ud
import functools
import operator
import re
import numpy as np
!pip install wordfreq
from wordfreq import word_frequency

consumer_key = "idjkP1aobw1UQd8xZ9RYiY5CZ"
consumer_secret = "jZFXsLJRtvR4pQvmuTJ94mnr1TJ0tYz1w4s0XI5TpR4U5tEnXe"
access_token = "1001251273981677568-5SxiGu3SisqPnzY3Zkq8QHh7vreYar"
access_token_secret = "XZn1rvLw10JnxJgKx05sW4eN0HqhaVjsasaqV5tEytsTu"
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)

api = tweepy.API(auth,wait_on_rate_limit=True)

In [246]:
tweetnum = 100

tbwTokenizer = TreebankWordTokenizer()
glove50Model = gensimapi.load("glove-wiki-gigaword-50")

tokenizedText = []
tknTextVector = []

def has_emoji(s):
    em_split_emoji = emoji.get_emoji_regexp().split(s)
    em_split_whitespace = [substr.split() for substr in em_split_emoji]
    em_split = functools.reduce(operator.concat, em_split_whitespace)
    emojiExists = False
    for emojiTest in em_split:
        if(emojiTest in UNICODE_EMOJI):
            emojiExists = True
    
    return emojiExists

def break_compound_word(compound_word,model):
    first_word = ""
    possible_words = []
    for i,xchar in enumerate(compound_word):
        first_word+=xchar
        try:
            model[first_word]
            second_word = compound_word[i+1:]
            
            print(first_word)
            print(second_word)
            print("---")
            model[second_word]
            possible_words.append([first_word,second_word])
        except:
            pass
        
    return possible_words

for tweet in tweepy.Cursor(api.search,q="#love",geocode="51.5098,-0.1180,1km",tweet_mode="extended").items(tweetnum):
    tweet_text = tweet._json["full_text"].lower()
    tweet_text_tokenized = tbwTokenizer.tokenize(tweet_text)
    tokenizedText.append(tweet_text_tokenized)
    tknVectorList = []
    ##Deal with emojis
    ##Deal with capital letters - ok
    ##Access the link, and if there is text, look for keywords using tf-idf
    ##Auto correct
    
    for text in tweet_text_tokenized:
        try:
            wordVector = glove50Model[text]
            tknVectorList.append(wordVector)
        except:
            solutionFound = False
            if(text[-1] == "."):
                try:
                    wordVector = glove50Model[text[:-1]]
                    tknVectorList.append(wordVector)
                    solutionFound = True
                except:
                    pass
            if(has_emoji(text)):
                em_split_emoji = emoji.get_emoji_regexp().split(text)
                em_split_whitespace = [substr.split() for substr in em_split_emoji]
                em_split = functools.reduce(operator.concat, em_split_whitespace)
                for eachEmoji in em_split:
                    emojiDetail = ud.name(eachEmoji)
                    emojiTokenized = tbwTokenizer.tokenize(emojiDetail.lower())
                    emojiWordVectors = []  
                    for descr in emojiTokenized:
                        try:
                            emojiVector = glove50Model[descr]
                            emojiWordVectors.append(emojiVector)
                        except:
                            pass
                    tknVectorList.extend(emojiWordVectors)
                solutionFound = True
                
            child_words = break_compound_word(text,glove50Model)
            highestfrq = 0
            if(len(child_words)!= 0):
                for child_word_set in child_words:
                    both_word_freq = word_frequency(child_word_set[0],"en")*word_frequency(child_word_set[1],"en")
                    if (both_word_freq > highestfrq):
                        most_likely_combo = child_word_set
                        highestfrq = both_word_freq
            
                child_word_vectors = []
                child_word_vectors.append(glove50Model[most_likely_combo[0]])
                child_word_vectors.append(glove50Model[most_likely_combo[1]])
                tknTextVectorList.extend(child_word_vectors)
                solutionFound = True
            
            if(not solutionFound):
                print("Failed to vectorize: " + text)
                print("    " + str(has_emoji(text)))
                    
    tknTextVector.append(tknVectorList)

TweepError: Failed to send request: Could not find a suitable TLS CA certificate bundle, invalid path: /usr/local/anaconda3/envs/hashtagrec/lib/python3.7/site-packages/certifi/cacert.pem

In [50]:
tknTextVector

(96,)

In [116]:
import unicodedata as ud
import emoji
import functools
import operator
import re

s = '\U0001F604\U0001F600\U0001F608'
# for c in s:
#     print('{} U+{:5X} {}'.format(c,ord(c),ud.name(c)))
    
def has_emoji(s):
    em_split_emoji = emoji.get_emoji_regexp().split(s)
    em_split_whitespace = [substr.split() for substr in em_split_emoji]
    em_split = functools.reduce(operator.concat, em_split_whitespace)
    emojiExists = False
    for emojiTest in em_split:
        if(emojiTest in UNICODE_EMOJI):
            emojiExists = True
    
    return emojiExists
            


In [118]:
testStringS = "hello🥰🥰🥰🥰🥰🥰"
len(testStringS)

11

In [83]:
testStringS[2]

'🥰'

In [119]:
boolean = has_emoji(testStringS)

boolean

True

In [148]:
from pattern.en import spelling


def reduce_lengthening(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)


word = "amazzziiing"
word_wlf = reduce_lengthening(word) #calling function defined above
print(word_wlf) #word lengthening isn't being able to fix it completely

correct_word = pattern.en.pattenspelling(word_wlf) 
print(correct_word)

ModuleNotFoundError: No module named 'pattern'

In [189]:



possible_words = break_compound_word("fearfull",glove50Model)

[['the', 'doggy']]

In [238]:
def break_compound_word(compound_word,model):
    first_word = ""
    possible_words = []
    for i,xchar in enumerate(compound_word):
        first_word+=xchar
        try:
            model[first_word]
            second_word = compound_word[i+1:]
            
            print(first_word)
            print(second_word)
            print("---")
            model[second_word]
            possible_words.append([first_word,second_word])
        except:
            pass
        
    return possible_words

wordset = break_compound_word("thedog",glove50Model)

sum_meaning = glove50Model[wordset[0][0]] + glove50Model[wordset[0][1]]

sum_meaning

t
hedog
---
th
edog
---
the
dog
---
thed
og
---


array([ 5.2807999e-01, -1.3812999e-01, -9.8856997e-01, -1.5544000e-01,
        1.0504800e+00,  4.9548301e-01, -1.5754800e+00, -5.8007997e-01,
        1.1497399e+00, -1.2244000e+00,  2.8232771e-01,  3.8110998e-01,
        8.8839948e-02,  6.1919999e-01,  4.7598049e-01, -1.7241201e-01,
        2.8213999e-01,  7.8604996e-01, -2.0422001e+00, -6.9960999e-01,
       -3.9665100e-01,  3.2700002e-03,  6.1680001e-01,  5.2424002e-01,
        2.6324999e-01, -3.6494000e+00, -1.8177600e+00,  5.2471101e-01,
       -2.3199916e-03, -8.7852997e-01,  5.5743999e+00,  9.0909988e-02,
       -1.1399500e+00,  3.2957000e-01, -7.6403871e-02,  3.7862489e-01,
        3.0857998e-01, -6.1034000e-01,  2.6602101e-01, -7.9814303e-01,
       -3.8490897e-01,  8.3189994e-02, -9.9576998e-01,  7.8985298e-01,
        8.0769002e-01, -3.2594001e-01,  4.5024902e-02, -1.0652900e+00,
        6.0065997e-01, -4.0061998e-01], dtype=float32)

array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
       -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
        2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
        1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
       -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
       -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
        4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
        7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
       -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
        1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01],
      dtype=float32)

[('gibralter', 0.7775185108184814),
 ('scragg', 0.7496911287307739),
 ('aptx', 0.7440819144248962),
 ('kamya', 0.724817156791687),
 ('mcnicol', 0.7228055596351624),
 ('clac', 0.7217633724212646),
 ('wildlings', 0.718341588973999),
 ('eniola', 0.7131443023681641),
 ('blout', 0.7082638740539551),
 ('sidhwa', 0.7071182727813721)]